In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import re
import time
import os
from selenium import webdriver

In [4]:
def random_user_agent():
    try:
        ua = UserAgent()
        return ua.random
    except:
        default_ua = 'Mozilla/5.0 (Macintosh; Intel Mac O21S X 10_12_3) \
                AppleWebKit/537.36 (KHTML, like Gecko) \
                Chrome/58.0.3029.110 Safari/537.36'
        return default_ua

def get_soup(url):
    
    headers = {'User-Agent': random_user_agent()}
    
    response = requests.get(url, headers=headers)
    results = response.content
    if not response.status_code == 404:
        soup = BeautifulSoup(results, 'lxml')
    return soup

def soup_attempts(url, total_attempts=5):

    soup = get_soup(url)

    if soup:
        return soup
    else:
        attempts = 0
        while attempts < total_attempts:
            time.sleep(3)
            soup = get_soup(url)
            if soup:
                return soup
        raise ValueError(f'FAILED to get soup for apt url {url}')

In [5]:
url = 'https://www.corcoran.com/nyc-real-estate/for-sale/williamsburg/30-bartlett-street/5151565'
soup = soup_attempts(url)

In [6]:
soup

<!DOCTYPE html>
<html lang="en"><head><meta content="width=device-width, initial-scale=1" name="viewport"/><meta charset="utf-8"/><meta content="text/html" http-equiv="Content-Type"/><link href="/static/favicon.ico" rel="shortcut icon" type="image/x-icon"/><link crossorigin="anonymous" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" rel="stylesheet"/><style data-styled="gpjKBA eqKSwt kQoTvr kpXCIj htePRI dmXTqa cpmDgc fhlStG kwBnei hMtQSt wGHFj ifAycJ bPsGgl bTOHhL cNzKMb iuzJKf dQWXck kbTfUs hDNToO jnhlJp eGKyFp eCJHeT dBYtGm cKZOWW djClTt eNUIut fBSNno haTtwu fRilmy jpRfwA jjJaTK knRzwQ eqDAxQ dfHEFZ gukWPa fPYvli bXPDOE fUAOTk fZuFhA vzVNT ipLvid bbVACy fshHlO iGDbgB bcnsrB kRBdIV jpnhrN cNdyRy boWTpJ fHVDfM biOaYE MQPVR dFdMal easBre dsufRg iuYYqP fMJIwl jNWojQ joMYOB fNRKtP jzdMOr joXjIr fnNQXj gcRYrv kskpme hnCopz buysat llDpXD bQMkgD bnZDmF jsLAET kzFvEv kBzUWz cfO